In [97]:
import numpy as np
import os
import plotly.graph_objects as go
from tqdm import tqdm
import pandas as pd

import chimera_fgo.util.general as general
from chimera_fgo.util.io import read_lidar_bin, read_gt
from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results, load_sv_positions
from chimera_fgo.util.geometry import euler_to_R
from chimera_fgo.odometry import odometry

import symforce
try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass 

import symforce.symbolic as sf
from chimera_fgo.symforce.tc_fgo import tc_fgo
from chimera_fgo.symforce.fgo_lidar_only import fgo_lidar_only

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True, precision=3)

Already set symforce epsilon
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [2]:
kitti_seq = '0027'
start_idx = 1550 if kitti_seq == '0028' else 0

gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

In [3]:
# Load satellite positions
sv_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'svs')
#svfile = os.path.join(sv_path, f'{kitti_seq}_saved_sats.csv')
sv_positions = load_sv_positions(gtpath, sv_path, kitti_seq, start_idx=start_idx)

# For all traces (0018, 0027, 0028, 0034), the number of satellites in the first 2000 frames is constant
N_SATS = len(sv_positions[0])

In [4]:
# Compute DOP
los = sv_positions[0]
los = los / np.linalg.norm(los, axis=1)[:,None]
A = np.hstack((los, np.ones((N_SATS, 1))))
Q = np.linalg.inv(A.T @ A)
PDOP = np.sqrt(Q[0,0] + Q[1,1] + Q[2,2])
Q

array([[ 0.304,  0.033,  0.138, -0.066],
       [ 0.033,  0.389,  0.048,  0.009],
       [ 0.138,  0.048,  2.138, -1.108],
       [-0.066,  0.009, -1.108,  0.688]])

### Parameters

In [5]:
TRAJLEN = 2000

MAX_BIAS = 200  # [m]
ATTACK_START = TRAJLEN // 2

# Sliding window of size 100 which shifts by 10 each iteration
N_SHIFT = 10
N_WINDOW = 200

GPS_RATE = 10  # ratio of odometry to GPS measurements

PR_SIGMA = 6

# Lidar odometry covariance
ODOM_R_SIGMA = 0.01  # 
ODOM_T_SIGMA = 0.05  # [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA

### Problem Setup 

In [6]:
# Lidar odometry
lidar_odom = [None] * (TRAJLEN - 1)
for i in range(TRAJLEN - 1):
    lidar_odom[i] = (lidar_Rs[i], lidar_ts[i])

### Spoofing

In [83]:
N_WINDOW = 200
MAX_BIAS = 200  # [m]

In [84]:
from scipy.stats import chi2

# Compute threshold
alpha = 0.001  # False alarm (FA) rate
T = chi2.ppf(1-alpha, df=N_SATS*(N_WINDOW/GPS_RATE))
print("Threshold = ", T)

traj_FA_rate = 1 - (1-alpha)**(TRAJLEN//GPS_RATE)

Threshold =  244.37046748942743


In [85]:
# Spoofed trajectory
spoofed_pos = gt_enu[:TRAJLEN].copy()

gps_spoofing_biases = np.zeros(TRAJLEN)  
gps_spoofing_biases[ATTACK_START:] = np.linspace(0, MAX_BIAS, TRAJLEN-ATTACK_START)  # Ramping attack

spoofed_pos[:,0] += gps_spoofing_biases
#spoofed_pos[:,1] += gps_spoofing_biases

# Spoofed ranges
PR_SIGMA = 6
spoofed_ranges = np.zeros((TRAJLEN, N_SATS))
for i in range(TRAJLEN):
    svs = sv_positions[i]
    for j in range(N_SATS):
        spoofed_ranges[i,j] = np.linalg.norm(spoofed_pos[i] - svs[j]) + np.random.normal(0, PR_SIGMA)

In [99]:
PR_SIGMA = 6

graph_positions = np.zeros((TRAJLEN, 3))
# graph_rotations = np.zeros((TRAJLEN, 3))
graph_attitudes = np.zeros((TRAJLEN, 3))
init_poses = [sf.Pose3.identity()] * N_WINDOW
init_poses[0] = sf.Pose3(sf.Rot3.from_rotation_matrix(gt_Rs[0]), sf.V3(gt_enu[0]))
e_ranges = np.zeros((N_WINDOW//GPS_RATE, N_SATS))
qs = np.zeros(TRAJLEN//GPS_RATE)

authentic = True
k_max = (TRAJLEN - N_WINDOW) // N_SHIFT + 1

for k in (pbar := tqdm(range(k_max + 1))):

    # For handling re-processing
    if not authentic: k = k - 1
    if k == k_max: break

    window = slice(N_SHIFT * k, N_SHIFT * k + N_WINDOW)
    odom = lidar_odom[window][:-1]

    if authentic:
        ranges = spoofed_ranges[window]
        satpos_enu = np.array(sv_positions[window])
        satpos = [[sf.V3(satpos_enu[i][j]) for j in range(N_SATS)] for i in range(N_WINDOW)]
        result = tc_fgo(init_poses, satpos, ranges, odom, PR_SIGMA, ODOM_SIGMA, GPS_RATE, fix_first_pose=True, debug=False)
    else:
        result = fgo_lidar_only(init_poses, odom, ODOM_SIGMA, fix_first_pose=True, debug=False)

    # Extract optimized positions and orientations
    window_positions = np.zeros((N_WINDOW, 3))
    window_attitudes = np.zeros((N_WINDOW, 3))
    for i in range(N_WINDOW):
        pose = result.optimized_values["poses"][i]
        window_positions[i] = pose.position().flatten()   
        window_attitudes[i] = pose.rotation().to_yaw_pitch_roll().flatten()   

    # Compute test statistic
    for i in range(N_WINDOW//GPS_RATE):
        svs = sv_positions[N_SHIFT * k + i * GPS_RATE]
        for j in range(N_SATS):
            e_ranges[i,j] = np.linalg.norm(window_positions[::GPS_RATE][i] - svs[j])
    q = np.sum((ranges[::GPS_RATE] - e_ranges)**2) / PR_SIGMA**2
    pbar.set_description(f"q = {q:.2f}")
    qs[k] = q

    # Check spoofing
    if q > T:
        if authentic:
            print("Attack detected at ", N_SHIFT * k)
            authentic = False
            # Skip saving this window - reprocess it next iteration
            #continue

            # Process the remainder of the trajectory with odometry only
            init_pose = (init_poses[0].R.to_rotation_matrix().to_numpy(), init_poses[0].t.to_numpy().flatten())
            odom = lidar_odom[N_SHIFT * k:]
            positions, rotations = odometry(init_pose, odom)
            graph_positions[N_SHIFT * k:] = positions
            break
            

    # Save trajectory
    graph_positions[N_SHIFT*k:N_SHIFT*(k+1)] = window_positions[:N_SHIFT]
    graph_attitudes[N_SHIFT*k:N_SHIFT*(k+1)] = window_attitudes[:N_SHIFT]

    # Update initial poses
    init_poses[:-N_SHIFT] = result.optimized_values["poses"][N_SHIFT:]

    # Use lidar odometry to initialize latest new poses
    n_pose = result.optimized_values["poses"][-1]
    n_pose = sf.Pose3(sf.Rot3.from_rotation_matrix(n_pose.R.to_rotation_matrix()), sf.V3(n_pose.t))
    for i in range(N_SHIFT):
        lidar_T = sf.Pose3(sf.Rot3.from_rotation_matrix(lidar_Rs[N_SHIFT*k + N_WINDOW + i]), sf.V3(lidar_ts[N_SHIFT*k + N_WINDOW + i]))
        n_pose = n_pose * lidar_T
        init_poses[-N_SHIFT+i] = n_pose

if authentic:
    # Save remaining poses in current window
    cutoff = N_SHIFT*k if authentic else N_SHIFT*(k+1)
    graph_positions[cutoff:] = window_positions[N_SHIFT:]
    graph_attitudes[cutoff:] = window_attitudes[N_SHIFT:]

q = 195.77:   5%|▌         | 10/182 [00:35<10:17,  3.59s/it]

In [95]:
init_poses[0].rotation().to_rotation_matrix()

array([[ 0.979,  0.16 ,  0.126],
       [-0.171,  0.981,  0.087],
       [-0.109, -0.106,  0.988]])

In [98]:
init_poses[0].t

array([[299.026],
       [  4.698],
       [-14.516]])

In [87]:
graph_positions_plot = graph_positions
gt_enu_plot = gt_enu[:TRAJLEN]
qs_plot = qs[:k_max]

In [88]:
# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=qs_plot, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=T*np.ones(len(qs_plot)), name='Threshold', line=dict(color='red', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
#fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.show()

In [89]:
fgo_traj = go.Scatter(x=graph_positions_plot[:,0], y=graph_positions_plot[:,1], hovertext=np.arange(TRAJLEN), name='FGO trajectory')
gt_traj = go.Scatter(x=gt_enu_plot[:,0], y=gt_enu_plot[:,1], hovertext=np.arange(TRAJLEN), name='Ground-truth')
spoof_traj = go.Scatter(x=spoofed_pos[:TRAJLEN,0], y=spoofed_pos[:TRAJLEN,1], hovertext=np.arange(OPT_TRAJLEN), name='Spoofed')
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[gt_traj, fgo_traj, spoof_traj, start])
fig.update_layout(width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
#fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)

fig.add_trace(go.Scatter(x=lidar_positions[:TRAJLEN,0], y=lidar_positions[:TRAJLEN,1], name='Lidar'))
fig.show()

In [ ]:
# Plot x and y errors over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,0] - gt_enu_plot[:,0], name='x error'))
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,1] - gt_enu_plot[:,1], name='y error'))
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,2] - gt_enu_plot[:,2], name='z error'))
# Plot RMSE
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=np.sqrt(np.mean((graph_positions_plot - gt_enu_plot)**2, axis=1)), name='RMSE'))
#fig.add_shape(type="line", x0=ATTACK_START, y0=-20, x1=ATTACK_START, y1=90, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Error [m]')
fig.update_layout(font=dict(size=15))
fig.show()


In [ ]:
from plotly.subplots import make_subplots

# Plot x, y, z errors in subplots on top of each other
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,0] - gt_enu_plot[:,0], name='x error'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,1] - gt_enu_plot[:,1], name='y error'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,2] - gt_enu_plot[:,2], name='z error'), row=3, col=1)
fig.update_layout(width=1200, height=700)
fig.update_layout(font=dict(size=15))
fig.show()

In [ ]:
# RMSE error
rmse_xyz = np.sqrt(np.mean((graph_positions_plot - gt_enu_plot)**2, axis=0))
print("RMSE (xyz): ", rmse_xyz)
print("RMSE (overall): ", rmse_xyz.mean())

In [ ]:
# Mean error
mean_error = np.mean((graph_positions_plot - gt_enu_plot), axis=0)
std_error = np.std((graph_positions_plot - gt_enu_plot), axis=0)
print("Mean error (xyz): ", mean_error)
print("Std error (xyz): ", std_error)

In [ ]:
# Rotational error
gt_attitudes_plot = gt_attitudes[:OPT_TRAJLEN]


In [22]:
p = {'x': 0, 'y': 0, 'z': 0}
np.savez_compressed('test.npz', **p)

In [24]:
results = np.load('test.npz')
results['x']

array(0)

In [75]:
from chimera_fgo.fgo import chimera_fgo

In [89]:
# ================================================= #
#                    PARAMETERS                     #
# ================================================= #
params = {}
params['MITIGATION'] = True
params['ATTITUDE'] = False

params['N_SHIFT'] = 10
params['N_WINDOW'] = 100
params['GPS_RATE'] = 10

params['TRAJLEN'] = 2000   

params['ATTACK_START'] = params['TRAJLEN'] // 2
params['ALPHA'] = 0.001  # False alarm (FA) rate

params['PR_SIGMA'] = 6.0  # [m]
params['PR_SPOOFED_SIGMA'] = 1e5  # [m]

# Lidar odometry covariance
ODOM_R_SIGMA = 0.01  # so(3) tangent space units
ODOM_T_SIGMA = 0.05  # [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA
params['ODOM_SIGMA'] = ODOM_SIGMA

# Attitude measurement standard deviations
ATT_SIGMA_SCALE = 0.2
ATT_SIGMA = ATT_SIGMA_SCALE * np.ones(3)  # so(3) tangent space units (~= rad for small values)
params['ATT_SIGMA'] = ATT_SIGMA

params['kitti_seq'] = '0018'
params['MAX_BIAS'] = MAX_BIAS

In [78]:
output = chimera_fgo(params)

Already set symforce epsilon
Running FGO on KITTI sequence 0018 with max bias of 200 meters
Loading data...


q = 154.44:  52%|█████▏    | 98/190 [02:07<01:58,  1.29s/it]

Attack detected at  970


q = 0.00: 100%|██████████| 190/190 [04:17<00:00,  1.35s/it] 
